In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [41]:
%%sql
drop view if exists seq;
create view seq as (
    select generate_series(1,30) as num
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.


[]

In [42]:
%%sql
select num,
    case 
        when mod(num,15) = 0 then 'fizzbuzz'
        when mod(num,5) = 0 then 'buzz'
        when mod(num,3) = 0 then 'fizz'
        else cast(num as varchar(4))
    end as sign
from seq

*  postgresql://padawan:***@db:5432/dsdojo_db
30 rows affected.


num,sign
1,1
2,2
3,fizz
4,4
5,buzz
6,fizz
7,7
8,8
9,fizz
10,buzz


In [43]:
%%sql
select
    s1.num,
    coalesce((s2.sign || s3.sign), s2.sign, s3.sign, cast(s1.num as varchar(10)))
from seq as s1
left join (
    select num, 'buzz' as sign
    from seq
    where num % 5 = 0
) as s2
on s1.num = s2.num
left join (
    select num, 'fizz' as sign
    from seq
    where num % 3 = 0
) as s3
on s1.num = s3.num

*  postgresql://padawan:***@db:5432/dsdojo_db
30 rows affected.


num,coalesce
1,1
2,2
3,fizz
4,4
5,buzz
6,fizz
7,7
8,8
9,fizz
10,buzz


In [44]:
%%sql
select
    s.num,
    coalesce(m.sign, cast(s.num as varchar(10))) as sign
from (select num,
        cast((num % 3 = 0) as integer)*1 + cast((num%5 = 0) as integer)*2 as idx
    from seq
) as s
left join (
    select idx, sign
    from (values (1,'fizz'),(2,'buzz'),(3,'fizzbuzz')) as tmp(idx,sign)
) as m
on s.idx = m.idx
order by s.num

*  postgresql://padawan:***@db:5432/dsdojo_db
30 rows affected.


num,sign
1,1
2,2
3,fizz
4,4
5,buzz
6,fizz
7,7
8,8
9,fizz
10,buzz
